# 1. Engine Room: Algebraic Dynamics

**Objective:** Validate the "From Scratch" NumPy engine and explore the mathematics of optimization.

In this notebook, we will:
1. **Verify Backpropagation:** Perform a Numerical Gradient Check to prove our calculus is correct.
2. **Compare Optimizers:** Pit SGD, Momentum, and Adam against each other on the MNIST dataset.
3. **Visualize Convergence:** See how adaptive learning rates (Adam) speed up the descent down the loss surface.

In [ ]:
# Setup: Add parent directory to path to import src
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

from src import numpy_engine as nn
from src.utils import set_seed
from src.data_loader import load_mnist
from src.visualization import plot_training_curves

# Ensure reproducibility
set_seed(42)

## Part 1: The Litmus Test (Numerical Gradient Checking)

Before training, we must prove our `backward()` methods are correct. We do this by comparing the **Analytical Gradient** (calculated by chain rule in our code) vs. the **Numerical Gradient** (calculated by finite differences).

$$ \frac{df}{dx} \approx \frac{f(x + \epsilon) - f(x - \epsilon)}{2\epsilon} $$

In [ ]:
def numerical_gradient_check(layer, x, epsilon=1e-5):
    """Approximates the gradient using finite differences."""
    # 1. Forward pass to get initial state
    _ = layer.forward(x)
    
    # 2. Backward pass to get Analytical Gradient
    # We simulate a gradient of 1.0 coming from the next layer for simplicity
    grad_output = np.ones_like(layer.forward(x))
    analytical_grad = layer.backward(grad_output)
    
    # 3. Compute Numerical Gradient for input x
    numerical_grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        orig_val = x[idx]
        
        # f(x + eps)
        x[idx] = orig_val + epsilon
        pos_out = np.sum(layer.forward(x))
        
        # f(x - eps)
        x[idx] = orig_val - epsilon
        neg_out = np.sum(layer.forward(x))
        
        # Central Difference
        numerical_grad[idx] = (pos_out - neg_out) / (2 * epsilon)
        
        # Reset
        x[idx] = orig_val
        it.iternext()
        
    # Compare
    diff = np.linalg.norm(analytical_grad - numerical_grad) / (np.linalg.norm(analytical_grad) + np.linalg.norm(numerical_grad))
    return diff

# Test on a Linear Layer
x_dummy = np.random.randn(5, 10) # Batch 5, Features 10
layer = nn.Linear(10, 5)

diff = numerical_gradient_check(layer, x_dummy)
print(f"Gradient Check Difference (Linear): {diff:.2e}")

assert diff < 1e-7, "Gradient check failed! Check your math."
print("\u2705 Gradient Check Passed!")

## Part 2: The Great Race (SGD vs Adam)

We will train a simple MLP on a subset of MNIST to visualize how different optimizers navigate the loss landscape.

In [ ]:
# Load small subset of MNIST for speed
x_train, y_train, x_test, y_test = load_mnist(train_size=2000, test_size=500)

def build_mnist_model():
    model = nn.NumpyMLP()
    model.add(nn.Linear(784, 64))
    model.add(nn.ReLU())
    model.add(nn.Linear(64, 32))
    model.add(nn.ReLU())
    model.add(nn.Linear(32, 10))
    return model

def train(model, optimizer, epochs=15):
    loss_fn = nn.CrossEntropyLoss()
    loss_history = []
    
    batch_size = 64
    num_batches = x_train.shape[0] // batch_size
    
    for epoch in range(epochs):
        epoch_loss = 0
        # Shuffle
        perm = np.random.permutation(x_train.shape[0])
        x_shuf = x_train[perm]
        y_shuf = y_train[perm]
        
        for i in range(num_batches):
            start = i * batch_size
            end = start + batch_size
            
            # 1. Forward
            preds = model.forward(x_shuf[start:end])
            
            # 2. Loss
            loss = loss_fn.forward(preds, y_shuf[start:end])
            epoch_loss += loss
            
            # 3. Backward
            grad = loss_fn.backward()
            model.backward(grad)
            
            # 4. Optimize
            optimizer.step()
            optimizer.zero_grad()
            
        loss_history.append(epoch_loss / num_batches)
        print(f"Epoch {epoch+1}: Loss {epoch_loss/num_batches:.4f}", end='\r')
    
    print(f"\nDone. Final Loss: {loss_history[-1]:.4f}")
    return loss_history

In [ ]:
# 1. SGD
print("Training SGD...")
model_sgd = build_mnist_model()
opt_sgd = nn.SGD(model_sgd.parameters(), lr=0.01)
loss_sgd = train(model_sgd, opt_sgd)

# 2. Momentum
print("Training Momentum...")
model_mom = build_mnist_model()
opt_mom = nn.Momentum(model_mom.parameters(), lr=0.01, momentum=0.9)
loss_mom = train(model_mom, opt_mom)

# 3. Adam
print("Training Adam...")
model_adam = build_mnist_model()
opt_adam = nn.Adam(model_adam.parameters(), lr=0.001)
loss_adam = train(model_adam, opt_adam)

## Part 3: Visualization

Notice how Adam converges significantly faster in the early epochs. This is due to the **Adaptive Moment Estimation**, which scales the learning rate individually for each parameter based on the variance of its gradients.

In [ ]:
history = {
    'SGD': loss_sgd,
    'Momentum': loss_mom,
    'Adam': loss_adam
}

plot_training_curves(history, title="Optimizer Benchmark (NumPy Engine)")